## Решение задачи регрессии с помощью ансамбля классификаторов. Значения таргета дискретизируются до целых чисел, после чего исходный датафрейм (ДФ) последовательно делится на n фолдов с помощью рекурсивной функции и для каждого шага обучается соответствующий классификатор. Деление происходит до тех пор, пока в каждом фолде остаётся одна цифра дискретного таргета. В текущей реализации n должно быть <= 9.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.preprocessing import StandardScaler

from sklearn import metrics

from lightgbm import LGBMRegressor, LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./data/boston.csv').sample(frac=1).reset_index(drop=True)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.08707,0.0,12.83,0,0.437,6.140,45.8,4.0905,5,398.0,18.7,386.96,10.27,20.8
1,0.33147,0.0,6.20,0,0.507,8.247,70.4,3.6519,8,307.0,17.4,378.95,3.95,48.3
2,0.13058,0.0,10.01,0,0.547,5.872,73.1,2.4775,6,432.0,17.8,338.63,15.37,20.4
3,3.83684,0.0,18.10,0,0.770,6.251,91.1,2.2955,24,666.0,20.2,350.65,14.19,19.9
4,3.16360,0.0,18.10,0,0.655,5.759,48.2,3.0665,24,666.0,20.2,334.40,14.13,19.9


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [4]:
target_name = 'MEDV'

In [5]:
# Перемешиваем датасет и разбиваем на train-test
df_train, df_test = train_test_split(df, test_size=0.3, shuffle=False)

# Масштабируем все столбцы кроме таргета и возвращаем всё в форму исходного датафрейма
scaler = StandardScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train.drop([target_name], axis=1)), columns = df_train.drop([target_name], axis=1).columns).join(df_train[target_name])
df_test_scaled = pd.DataFrame(scaler.transform(df_test.drop([target_name], axis=1)), columns = df_test.drop([target_name], axis=1).columns).join(df_test[target_name].reset_index(drop=True))

In [6]:
# Сортировка датасета по значению таргета
df_train_scaled_sorted = df_train_scaled.sort_values(by=target_name).reset_index(drop=True)

In [7]:
# Функция деления датафрейма на n примерно равных частей (границы разбивки сдвигаются к ближайшим целым значениям таргета)

def split_n_folds(ser_target, n):
    # Вычисляем количество "цифр" (дискретных значений) таргета в ser_target (объект Series с таргет-столбцом из рассматриваемого сегмента основного ДФ)
    digits = ser_target.astype('int8').unique()
    digits_numb = len(digits)

    # Если количество цифр меньше, чем n, уменьшаем значение n до количества цифр
    if digits_numb < n:
        n = int(digits_numb)

    # Возвращаемый массив индексов для каждого fold, на которые разбивается ser_target
    ind_arr = {}
    start_ind = ser_target.index.min()
    last_ind = start_ind

    for i in range(n):
        # Вычисление последнего индекса текущего fold (на основе общего количества записей в ser_target)
        cur_end_1 = start_ind + len(ser_target) // n * (i + 1) - 1
        # Вычисление последнего индекса текущего fold (на основе количества оставщихся цифр таргета)
        cur_end_2 = ser_target[ser_target < digits[-(n - i - 1)]].index.max()
        # Выбор наименьшего индекса из двух
        cur_end = min(cur_end_1, cur_end_2)
        # Если последний индекс оказался меньше начального, сдвигаем последний до начального (вправо)
        if cur_end < last_ind:
            cur_end = last_ind
        
        # Определяем, на какую цифру таргета указывает последний индекс текущей части
        cur_target_end_digit = np.floor(ser_target[cur_end])
        
        # Если текущий fold не является последним, то сдвигаем его последний индекс вправо до границы текущей цифры таргета
        if i < (n-1):
            cur_end = ser_target[ser_target < cur_target_end_digit + 1].index.max()
        # Если текущий fold - последний - сдвигаем его последний индекс до последнего индекса ser_target
        else:
            cur_end = ser_target.index.max()
        
        # Запись полученного списка индексов в возвращаемый массив
        ind_arr[i] = ser_target.loc[last_ind:(cur_end)].index
        
        last_ind = cur_end + 1
    return ind_arr

In [8]:
df_train_scaled_sorted.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,3.652516,-0.478537,0.997496,-0.26968,1.21195,-1.199290,1.107308,-1.078561,1.689130,1.544409,0.811431,0.455364,2.556201,5.0
1,6.777592,-0.478537,0.997496,-0.26968,1.21195,-0.862345,1.107308,-1.108624,1.689130,1.544409,0.811431,0.330073,1.474773,5.0
2,2.246269,-0.478537,0.997496,-0.26968,1.21195,-0.416993,1.107308,-1.032108,1.689130,1.544409,0.811431,0.455364,2.013356,5.6
3,-0.381412,-0.478537,2.393386,-0.26968,0.46734,-1.256424,1.045978,-0.954094,-0.640251,1.816203,0.764434,-0.099674,1.615458,7.0
4,4.434002,-0.478537,0.997496,-0.26968,1.21195,-2.567577,1.107308,-0.999610,1.689130,1.544409,0.811431,-2.785910,3.464260,7.0


In [9]:
# Создание промежуточного целочисленного таргета для работы классификатора

df_train_scaled_sorted['class'] = 0
df_train_scaled_sorted.astype({'class': 'int8'})

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,class
0,3.652516,-0.478537,0.997496,-0.269680,1.211950,-1.199290,1.107308,-1.078561,1.689130,1.544409,0.811431,0.455364,2.556201,5.0,0
1,6.777592,-0.478537,0.997496,-0.269680,1.211950,-0.862345,1.107308,-1.108624,1.689130,1.544409,0.811431,0.330073,1.474773,5.0,0
2,2.246269,-0.478537,0.997496,-0.269680,1.211950,-0.416993,1.107308,-1.032108,1.689130,1.544409,0.811431,0.455364,2.013356,5.6,0
3,-0.381412,-0.478537,2.393386,-0.269680,0.467340,-1.256424,1.045978,-0.954094,-0.640251,1.816203,0.764434,-0.099674,1.615458,7.0,0
4,4.434002,-0.478537,0.997496,-0.269680,1.211950,-2.567577,1.107308,-0.999610,1.689130,1.544409,0.811431,-2.785910,3.464260,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,-0.246133,-0.478537,1.211803,-0.269680,0.431882,1.783401,0.775406,-0.853181,-0.523782,-0.044070,-1.773412,0.219381,-1.544983,50.0,0
350,0.290273,-0.478537,0.997496,3.708099,0.662357,1.090468,1.017117,-1.213049,1.689130,1.544409,0.811431,0.404428,-1.370193,50.0,0
351,0.198454,-0.478537,0.997496,3.708099,0.662357,0.602631,0.991864,-1.140794,1.689130,1.544409,0.811431,0.228833,-1.260771,50.0,0
352,-0.188340,-0.478537,1.211803,-0.269680,0.431882,2.427991,0.970218,-0.818061,-0.523782,-0.044070,-1.773412,0.165505,-1.265034,50.0,0


In [10]:
# Функция вычисления разницы между максимальной и минимальной цифрой таргета

def borders_diff(df):
    border_min = np.floor(df.MEDV.min())
    border_max = np.floor(df.MEDV.max())

    return (border_max - border_min)

In [11]:
# Рекурсивная функция для деления текущего сегменда датафрейма на n folds, вычисления нового таргета class и обучения классификаторов

def split_recursion(df, n, cl):
    # Получение массива индексов для каждого fold после деления
    i_arr = split_n_folds(df[target_name], n)

    # Проставляем новый таргет (class) для train (работает для не более 9-ти классов)
    for i in range(len(i_arr)):
        class_cur = cl

        # Перевычисление нового таргета (методом добавления цифры к старому class)
        class_new = int(class_cur * 10 + (i + 1))
        df.loc[i_arr[i], 'class'] = class_new
        
        # Выделением текущего fold в новый ДФ
        df_cur = df.loc[i_arr[i], :]
        
        # Если в новом ДФ более одной цифры таргета, запускаем текущую функцию снова для дальнейшего деления
        if borders_diff(df_cur) >= 1:
            split_recursion(df_cur, n, class_new)
        # Иначе проверка и дополнение массива для пересчёта нового таргета в старый
        else:
            border_min = np.floor(df_cur.MEDV.min())
            border_max = np.floor(df_cur.MEDV.max())
            if border_min != border_max:
                print("Error: border_min != border_max")

            target_class_arr[class_new] = border_min + 0.5
                
    # Создание и обучение классификатора на текущем сегменте ДФ с новым таргетом (class)
    lgbm_clf_arr[class_cur] = LGBMClassifier()
    lgbm_clf_arr[class_cur].fit(df.drop([target_name, 'class'], axis=1), df['class'])

In [33]:
# Задание количества folds для разбиения на каждом шаге
N = 7

# Массив для пересчёта нового таргета в старый
target_class_arr = {}
# Массив для обученных классификаторов
lgbm_clf_arr = {}

# Запуск основной функции, которая заполняет созданные выше массивы
split_recursion(df_train_scaled_sorted, N, 0)

In [34]:
#df_train_arr.keys()

In [35]:
#lgbm_clf_arr

In [36]:
# Предсказание таргета (старого) для текущего ряда
def predict_row(df_row, cl):
    if cl in lgbm_clf_arr:
        pred = lgbm_clf_arr[cl].predict(df_row.reshape(1, -1))[0]
        class_result = predict_row(df_row , pred)
    else:
        return target_class_arr[cl]
    
    return class_result

# Предсказание таргета (старого) для всего датасета
def itertuples_func(df):
    return np.array([predict_row(np.array(row[0:-1]), 0) for row in df.itertuples(index=False)])

In [37]:
# Функция вычисления метрики MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Функция для вычисления всех метрик
def dataframe_metrics(y_test,y_pred):
    stats = [
       metrics.mean_absolute_error(y_test, y_pred),
       np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
       metrics.r2_score(y_test, y_pred),
       mean_absolute_percentage_error(y_test, y_pred)
    ]
    return stats

measured_metrics = pd.DataFrame({"error_type":["MAE", "RMSE", "R2", "MAPE"]})
measured_metrics.set_index("error_type")

""
error_type
MAE
RMSE
R2
MAPE


In [38]:
# Вычисление метрик на основе обученных классификаторов

measured_metrics["lgbm_clf"] = dataframe_metrics(df_test_scaled[target_name], itertuples_func(df_test_scaled))
measured_metrics

,error_type,lgbm_clf
0,MAE,3.003289
1,RMSE,4.612362
2,R2,0.732337
3,MAPE,14.514156


In [39]:
# Вычисление метрик на основе стандартного регрессора

lgbm_reg = LGBMRegressor()

lgbm_reg.fit(df_train_scaled.drop([target_name], axis=1), df_train_scaled[target_name])

measured_metrics["lgbm_reg"] = dataframe_metrics(df_test_scaled[target_name], lgbm_reg.predict(df_test_scaled.drop([target_name], axis=1)))
measured_metrics

,error_type,lgbm_clf,lgbm_reg
0,MAE,3.003289,2.583419
1,RMSE,4.612362,3.727309
2,R2,0.732337,0.825204
3,MAPE,14.514156,12.738674
